# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [63]:
# load necessary packages
import re
import Bio.SeqIO

In [64]:
# Read FastQ file and convert to string
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_str = []
for seqrecord in seqreads:
    seqreads_str.append(str(seqrecord.seq))

In [65]:
#Creates a function for creating the reverse compliment of a sequence
def reverse_complement(seq, unk_partner = 'N'):
    """Get reverse complement of a DNA sequence.
    
    Parameters
    -----------
    seq : str
        Uppercase DNA sequence.
        
    Returns
    -------
    str
        Reverse complement of the sequence in upper case.
        
    Example
    --------
    >>> reverse_complement('ATGCAC')
    'GTGCAT'
    
    """
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    rseq = ''
    for a in seq:
        if a in base_partner:
            # look up the complementary base in the dictionary
            pair = base_partner[a]
            rseq = pair + rseq
        else:
            rseq = unk_partner + rseq
    return rseq

In [66]:
#define a function to read all HA associated barcodes
def read_barcodeHA(seqread, bclen, upstream='AGGCGGCCGC', HAend='CCGGATTTGCATATAATGATGCACCAT'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
    Example
    -------
    >>> read_barcode('TTTTTTTTTTTTTTTTGCGGCCGCCT', bclen=16)
    'AAAAAAAAAAAAAAAA'
        
    """
    barcodeHA_re= re.compile(HAend + upstream + '(?P<barcode>[ATCG]{'+ str(bclen) +'})') #compiles list of all HA associated barcodes
    listHA=[] 
    for seq in seqreads:
        seq = seq.upper() #converts sequence to Uppercase
        seq = reverse_complement(seq) #Creates Reverse complement
        matchHA = barcodeHA_re.search(seq) # finds HA associated sequences
        if matchHA: 
            barcode = matchHA.group("barcode")
            listHA.append(str(barcode)) #creates a string with all matching barcodes
    return listHA

In [67]:
#Define a function to read all NA associated barcodes
def read_barcodeNA(seqread, bclen, upstream='AGGCGGCCGC', NAend='CACGATAGATAAATAATAGTGCACCAT'): 
    barcodeNA_re= re.compile(NAend + upstream + '(?P<barcode>[ATCG]{'+ str(bclen) +'})') #compiles list of all NA associated barcodes
    listNA=[] 
    for seq in seqreads:
        seq = seq.upper() #Upper case
        seq = reverse_complement(seq) #Reverse compliment
        matchNA = barcodeNA_re.search(seq) # Finds NA associated reads
        if matchNA:
            barcode = matchNA.group("barcode")
            listNA.append(str(barcode))#creates a string with all matching barcodes
    return listNA

In [68]:
#Dictionary plus counts of HA barcodes and NA barcodes
BarlistHA= read_barcodeHA(seqreads_str, bclen=16) # calls HA barcode function
BarlistNA= read_barcodeNA(seqreads_str, bclen=16) # calls NA barcode function
BarHACount={}
BarNACount={} #initiate empty dictionaries
for string in BarlistHA: #focusing on HA first
    if string in BarHACount: #adds 1 to count if barcode exists
        BarHACount[string]+=1
    else: #creates new entry with the value of 1
        BarHACount[string]=1
for string in BarlistNA: #creating NA dictionary now
    if string in BarNACount: #adds 1 to existing barcode
        BarNACount[string]+=1
    else: #creates new entry with a value of 1
        BarNACount[string]=1

In [71]:
# With Established dictionaries we can sum values
HAsum= sum(BarHACount.values())
NAsum= sum(BarNACount.values())
print("Total reads mapped to HA are", (HAsum))
print("Total reads mapped to NA are", (NAsum))

Total reads mapped to HA are 5249
Total reads mapped to NA are 3909


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [74]:
#Use the max function to find the largest value in our dictionaries
HAnum=max(BarHACount.values())
LongestHABar=max(BarHACount, key=BarHACount.get)
NAnum=max(BarNACount.values())
LongestNABar=max(BarNACount, key=BarNACount.get)
print('The HA barcode', LongestHABar, 'has the most counts:',HAnum)
print('The NA barcode', LongestNABar, 'has the most counts:',NAnum)

The HA barcode CCCGACCCGACATTAA has the most counts: 155
The NA barcode ACCAGTTCTCCCCGGG has the most counts: 152
